### Mini Projeto

In [23]:
!pip install tweepy
!pip install pymongo[srv]
!pip install dnspython

### Criando conexao com Twitter

In [17]:
# twitter credentials and tools
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from datetime import datetime
import json

credentials = {}
credentials['CONSUMER_KEY'] = 'mvqEjAxROjNDyZuGrM0Q4v6xU'
credentials['CONSUMER_SECRET'] = 'T1lOWzwgE140tfSNpySWIRqP9pMvsGu1OiN5GGMVygQ8mNRTYk'
credentials['ACCESS_TOKEN'] = '15453963-cbftpzokETSt1GCjkB7aYzElpYtWGuMm8lwVdq65e'
credentials['ACCESS_SECRET'] = 'joP73kijFoWROw6es4uMa3s3snpHH0lPgqnbLQS6fqLru'

#Gravando as credenciais em um arquivo
with open("twitter_credentials.json", "w") as file:
    json.dump(credentials, file)



In [10]:
# Carregando as credenciais salvas no arquivo na variavel creds
with open("twitter_credentials.json", "r") as file:
    creds = json.load(file)
    
print(creds)

{'CONSUMER_KEY': 'mvqEjAxROjNDyZuGrM0Q4v6xU', 'CONSUMER_SECRET': 'T1lOWzwgE140tfSNpySWIRqP9pMvsGu1OiN5GGMVygQ8mNRTYk', 'ACCESS_TOKEN': '15453963-cbftpzokETSt1GCjkB7aYzElpYtWGuMm8lwVdq65e', 'ACCESS_SECRET': 'joP73kijFoWROw6es4uMa3s3snpHH0lPgqnbLQS6fqLru'}


In [8]:
# Criando as chaves de autenticacao para a aplicacao ter permissao de acessi
auth = OAuthHandler(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])

auth.set_access_token(creds['ACCESS_TOKEN'], creds['ACCESS_SECRET'])

In [11]:
# Criando classe para capturar o stream de dados do twitter e armazenar no bd
class MyListener(StreamListener):
    def on_data(self, dados):
        tweet = json.loads(dados)
        created_at = tweet['created_at']
        id_str = tweet['id_str']
        text = tweet['text']
        obj = {"created_at":created_at, "id_str":id_str, "text":text}
        tweetind = col.insert_one(obj).inserted_id
        print(obj)
        return True
        
        

In [13]:
# Criando o objeto myListener
myListener = MyListener()

In [14]:
# Criando o objeto myStream
myStream = Stream(auth, listener = myListener)

### Criando conexao com o MongoDB (no MongoDB Atlas)

In [31]:
# Conectando no mongodb
from pymongo import MongoClient

In [32]:
client = MongoClient('localhost', 27017)

In [33]:
# Criando o banco twitterdb
db = client.twitterdb

# Criando a collection tweets
col = db.tweets

In [34]:
# Definindo as palavras chaves para busca de tweets
keywords = ['Big Data', 'Python', 'Data Mining', 'Data Science']

In [39]:
# Iniciando a coleta dos tweets
myStream.filter(track=keywords)

{'created_at': 'Tue Feb 11 10:26:42 +0000 2020', 'id_str': '1227177132062904321', 'text': 'RT @karen_ec_levy: 📢Early career folks! Cornell\'s new Center for Data Science is sponsoring 3-year "research assistant professorships" (aki…', '_id': ObjectId('5e4281684631b55482f7e508')}
{'created_at': 'Tue Feb 11 10:26:45 +0000 2020', 'id_str': '1227177143198720000', 'text': 'RT @parscale: Big Data from New Hampshire:\n\n✅ 52,559 Tickets\n\n✅ 24,732 Voters Identified (41% From NH)\n\n✅ 17% Didn’t Vote in 2016\n\n✅ 25.4%…', '_id': ObjectId('5e42816a4631b55482f7e509')}
{'created_at': 'Tue Feb 11 10:26:46 +0000 2020', 'id_str': '1227177146520567808', 'text': 'RT @migrpolcentre: Apply for our executive training on #childprotection challenges, April 28-30.\n@Sam_Okyere_ leads a session on ethical da…', '_id': ObjectId('5e42816b4631b55482f7e50a')}
{'created_at': 'Tue Feb 11 10:26:46 +0000 2020', 'id_str': '1227177147325800448', 'text': '@colonus_css CS:GOでは基本的にUserMessagを使ったメッセージしか使えないですね。\nhttps://

KeyboardInterrupt: 

In [40]:
myStream.disconnect()

In [41]:
# Verificando se tem dados no mongodb
col.find_one()

{'_id': ObjectId('5e4280b54631b55482f7e4f8'),
 'created_at': 'Tue Feb 11 10:23:44 +0000 2020',
 'id_str': '1227176383358656512',
 'text': 'RT @dataeconomy: Finland, Norway, Sweden, Iceland and Denmark. The big five are the crown jewels of the #datacentre ecosphere, and together…'}

### Iniciando a analise dos dados com Pandas e SciKit-Learn

In [48]:
# Criando um dataset

# Aqui eu estou criando uma linha no dataset para cara registro encontrado no banco.
# Para isso eu utilizei o List Comprehension
dataset = [{"created_at": item["created_at"], "text": item["text"],} for item in col.find()]

In [49]:
import pandas as pd

In [50]:
# Criando o dataframe a partir do dataset
df = pd.DataFrame(dataset)

In [52]:
# Imprimindo o dataframe
df

,created_at,text
0,Tue Feb 11 10:23:44 +0000 2020,"RT @dataeconomy: Finland, Norway, Sweden, Icel..."
1,Tue Feb 11 10:23:44 +0000 2020,RT @josephfcox: New: here is how popular email...
2,Tue Feb 11 10:23:45 +0000 2020,@IveMarx @destandaard Dat vind ik een oversimp...
3,Tue Feb 11 10:23:46 +0000 2020,RT @hardmaru: SciPy 1.0: fundamental algorithm...
4,Tue Feb 11 10:23:49 +0000 2020,Get number of Null values in a #pandas datafra...
5,Tue Feb 11 10:23:52 +0000 2020,"smicallef /spiderfoot: SpiderFoot, the most co..."
6,Tue Feb 11 10:23:54 +0000 2020,"RT @mvattuone: scene from werner herzog's ""pro..."
7,Tue Feb 11 10:23:57 +0000 2020,RT @hardmaru: SciPy 1.0: fundamental algorithm...
8,Tue Feb 11 10:23:57 +0000 2020,RT @RatanSharda55: Very brave to stand by your...
9,Tue Feb 11 10:23:58 +0000 2020,RT @ayatokura: 和歌山県主催のきのくにICTプログラミングコンテストの最優秀賞...


In [53]:
# Importando o scikit-learn
from sklearn.feature_extraction.text import CountVectorizer

In [54]:
# Usando o CountVectorizer para criar uma matriz de documentos
cv = CountVectorizer()
count_matrix = cv.fit_transform(df.text)

In [58]:
# Contando o numero de ocorrencia das principais palavras em nosso dataset
word_count = pd.DataFrame(cv.get_feature_names(), columns=["word"])
word_count["count"] = count_matrix.sum(axis=0).tolist()[0]
word_count = word_count.sort_values("count", ascending=False).reset_index(drop=True)

word_count[:50]

,word,count
0,rt,26
1,https,24
2,co,24
3,and,20
4,in,19
5,data,18
6,big,15
7,python,14
8,the,13
9,new,10
